In [1]:
import pandas as pd
import numpy as np

## CSV Importation and transformation

We import the csv file

In [2]:
df = pd.read_csv('DEMOCRATIE_ET_CITOYENNETE.csv')
df.head(2)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,reference,title,createdAt,publishedAt,updatedAt,trashed,trashedStatus,authorId,authorType,...,QUXVlc3Rpb246MTM1 - Quel pourrait être le rôle de chacun pour faire reculer les incivilités dans la société ?,QUXVlc3Rpb246MTM2 - Quelles sont les discriminations les plus répandues dont vous êtes témoin ou victime ?,QUXVlc3Rpb246MTM3 - Que faudrait-il faire pour lutter contre ces discriminations et construire une société plus solidaire et plus tolérante ?,QUXVlc3Rpb246MTM4 - Pensez-vous qu'il faille instaurer des contreparties aux différentes allocations de solidarité ?,"QUXVlc3Rpb246MTM5 - Si oui, lesquelles ?","QUXVlc3Rpb246MTQx - Que pensez-vous de la situation de l'immigration en France aujourd'hui et de la politique migratoire ? Quelles sont, selon vous, les critères à mettre en place pour définir la politique migratoire ?","QUXVlc3Rpb246MTQy - En matière d'immigration, une fois nos obligations d'asile remplies, souhaitez-vous que nous puissions nous fixer des objectifs annuels définis par le Parlement ?",QUXVlc3Rpb246MTQz - Que proposez-vous afin de répondre à ce défi qui va durer ?,"QUXVlc3Rpb246MTQ0 - Quelles sont, selon vous, les modalités d'intégration les plus efficaces et les plus justes à mettre en place aujourd'hui dans la société ?",QUXVlc3Rpb246MTQ1 - Y a-t-il d'autres points sur la démocratie et la citoyenneté sur lesquels vous souhaiteriez vous exprimer ?
0,UHJvcG9zYWw6MWVjMTkxYjAtMWUyMS0xMWU5LTk0ZDItZm...,1-2,Les augmentations de rémunérations,2019-01-22 09:38:41,2019-01-22 09:38:41,NaN,False,NaN,VXNlcjo4Mjc4NzQxYS0xZTFkLTExZTktOTRkMi1mYTE2M2...,Citoyen / Citoyenne,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afin d’éviter de creuser les inégalités ne plu...
1,UHJvcG9zYWw6NTA4N2I3ZDEtMWUyMS0xMWU5LTk0ZDItZm...,1-3,rénover l'enquête publique pour en faire un vr...,2019-01-22 09:40:04,2019-01-22 09:40:04,2019-01-22 09:41:00,False,NaN,VXNlcjo4OWQ3MzE5My0xZDYwLTExZTktOTRkMi1mYTE2M2...,Citoyen / Citoyenne,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We correct the date to remove the HH:MM part

In [3]:
def date_curation (date):
    try: 
        return(date[:10])
    except TypeError:
        return(None)

df['createdAt']=df['createdAt'].apply(date_curation)
df['publishedAt']=df['createdAt'].apply(date_curation)
df['updatedAt']=df['updatedAt'].apply(date_curation)

We change the structure of the file to have one row per question answered

In [4]:
df_elastic = pd.DataFrame()
colnumber = 11
for question in (df.columns[colnumber:]):
    df_temp = df.iloc[:,[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, colnumber]].copy().rename(columns={question : 'answer'})
    df_temp["question"]=question[20:]
    df_elastic = pd.concat([df_elastic, df_temp])
    print (question[20:])
    print(colnumber)
    colnumber +=1
df_elastic = df_elastic.reset_index()

En qui faites-vous le plus confiance pour vous faire représenter dans la société et pourquoi ?
11
En dehors des élus politiques, faut-il donner un rôle plus important aux associations et aux organisations syndicales et professionnelles ?
12
Si oui, à quel type d'associations ou d'organisations ? Et avec quel rôle ?
13
Que faudrait-il faire pour renouer le lien entre les citoyens et les élus qui les représentent ?
14
Le non-cumul des mandats instauré en 2017 pour les parlementaires (députés et sénateurs) est :
15
Pourquoi ?
16
Que faudrait-il faire pour mieux représenter les différentes sensibilités politiques ?
17
Pensez-vous qu'il serait souhaitable de réduire le nombre d'élus (hors députés et sénateurs) ?
18
Si oui, lesquels ?
19
Que pensez-vous de la participation des citoyens aux élections et comment les inciter à y participer davantage ?
20
Faut-il prendre en compte le vote blanc ?
21
Si oui, de quelle manière ?
22
Que faudrait-il faire aujourd'hui pour mieux associer les citoyens

## Sentiment analysis component using textblob

In [7]:
from textblob import TextBlob
from textblob_fr import PatternTagger, PatternAnalyzer

In [8]:
def sentiment_analysis(expression):
    blob = TextBlob(expression, pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())
    return (blob.sentiment[0], blob.sentiment[1])

In [9]:
df_elastic['answer'] = df_elastic['answer'].fillna("")
df_elastic['positivity'], df_elastic['intensity'] = zip(*df_elastic['answer'].map(sentiment_analysis))

In [8]:
df_elastic['sentiment'] = df_elastic['positivity']*df_elastic['intensity']

In [10]:
df_elastic.to_csv("democratie_enriched.csv", encoding='utf-8', index=False)

## Sentiment analysis component using translation and flair

### Translation

In [ ]:
import pandas as pd
import time
df_elastic = pd.read_csv('democratie_enriched.csv', encoding='utf-8', engine='python')
df_elastic['answer'] = df_elastic['answer'].fillna("")

Tried to do it using googletrans, free API that uses google Translate API, but I got kicked by Google :/ even with a VPN, using my phone as a modem

```python
errors = pd.read_csv('errors.csv')
batch_size = 10
alert_i = errors["alert_i"]
alert_min = errors["alert_min"]
alert_max = errors["alert_max"]
warning = 0
for i in range(int(errors.iloc[-1]['alert_max']/batch_size), int((len(df_elastic))/batch_size)):
    minim = i*batch_size
    maxim = min((i+1)*batch_size, len(df_elastic))
    mini = df_elastic.iloc[minim:maxim][['answer']].copy()
    concat = ''
    for row in mini['answer']:
        if row =='':
            row = '0000'
        concat= concat+row+'\n\n\n'
    
    from googletrans import Translator

    translator = Translator()
    try:
        result = translator.translate(concat, src = 'fr', dest = 'en').text
        warning = 0
        df_elastic.translation.iloc[minim:maxim] = result.split("\n\n\n")
    except:
        if warning ==1 :
            break
        print("raise error for {i}, {minim}".format(i=i,minim=minim))
        alert_i=alert_i.append(pd.Series(i)).reset_index(drop=True)
        alert_max=alert_max.append(pd.Series(maxim)).reset_index(drop=True)
        alert_min=alert_min.append(pd.Series(minim)).reset_index(drop=True)
        warning = 1
        pass
    
    if (maxim%2000==0):
        print("{i}, {maxim}".format(i=i,maxim=maxim))
```

### Flair part

To be done once the translation part is ok

## Geocoding and source adding

geocoding is a file that has been downloaded using http://www.nosdonnees.fr/wiki/index.php/Image:EUCircos_Regions_departements_circonscriptions_communes_gps.csv.gz

In [2]:
df_elastic_clean = pd.read_csv('democratie_enriched.csv', encoding='utf-8', engine='python')

In [4]:
try:
    df_elastic = df_elastic_clean.drop(['departement','city','latitude','longitude'], axis=1)
except:
    pass
def code_correction(code):
    if len(code)==4:
        return(str(code[:-2]+'000'))
    if len(code)==6:
        return(str('0'+code[:-2]))
    return(str(code[:-2]))

df_elastic['authorZipCode']=df_elastic['authorZipCode'].astype(str)
df_elastic['authorZipCode']=df_elastic['authorZipCode'].apply(code_correction)

In [5]:
geocoding = pd.read_csv('geocoding_corrected.csv', sep=';')

geocoding['codes_postaux']=geocoding['codes_postaux'].astype(str)

def code_correction(code):
    if len(code)==2:
        return(code+'000')
    if len(code)==4:
        return('0'+code)
    return(code)

geocoding['codes_postaux']=geocoding['codes_postaux'].apply(code_correction)

df_elastic['authorZipCode']=df_elastic['authorZipCode'].astype(str)
geocoding= (geocoding[["nom_departement","nom_commune","codes_postaux","latitude","longitude"]]
            .sort_values(by=["codes_postaux"], axis=0)
            .drop_duplicates(["codes_postaux"], 'first')
            .rename(columns={"nom_departement":"departement", "nom_commune":"city"}))
geocoding=geocoding.fillna(method='pad')
df_elastic = pd.merge(df_elastic, geocoding, how='left', left_on = 'authorZipCode', right_on='codes_postaux')

In [6]:
df_elastic.tail()

,index,id,reference,title,createdAt,publishedAt,updatedAt,trashed,trashedStatus,authorId,...,question,positivity,intensity,sentiment,source,departement,city,codes_postaux,latitude,longitude
2739993,74049,UHJvcG9zYWw6ODQzY2YxMDctNDE4YS0xMWU5LWJmNTYtZm...,1-89234,UNE DEMOCRATIE EQUITABLE,2019-03-08,2019-03-08,NaN,False,NaN,VXNlcjo0ZWQ2YmFiMS0zZDk0LTExZTktYmY1Ni1mYTE2M2...,...,Y a-t-il d'autres points sur la démocratie et ...,0.0,0.0,0.0,Grand debat - democratie et citoyennete,NaN,NaN,NaN,NaN,NaN
2739994,74050,UHJvcG9zYWw6OTJlMmQ4MzctNDE4YS0xMWU5LWJmNTYtZm...,1-89235,Pragmatiqme,2019-03-08,2019-03-08,NaN,False,NaN,VXNlcjplMmIyNzljZi00MTgxLTExZTktYmY1Ni1mYTE2M2...,...,Y a-t-il d'autres points sur la démocratie et ...,0.0,0.0,0.0,Grand debat - democratie et citoyennete,NaN,NaN,NaN,NaN,NaN
2739995,74051,UHJvcG9zYWw6YjEzNmQ1YTItNDE4YS0xMWU5LWJmNTYtZm...,1-89236,Revision totale,2019-03-08,2019-03-08,NaN,False,NaN,VXNlcjoxMTRmNzk1OC00MTgyLTExZTktYmY1Ni1mYTE2M2...,...,Y a-t-il d'autres points sur la démocratie et ...,0.0,0.0,0.0,Grand debat - democratie et citoyennete,NaN,NaN,NaN,NaN,NaN
2739996,74052,UHJvcG9zYWw6YmQzZDliY2EtNDE4YS0xMWU5LWJmNTYtZm...,1-89238,Trop d'élus qui ne servent à rien!,2019-03-08,2019-03-08,NaN,False,NaN,VXNlcjoxOWZhYzQ1Yi0xZmQ2LTExZTktOTRkMi1mYTE2M2...,...,Y a-t-il d'autres points sur la démocratie et ...,0.0,0.0,0.0,Grand debat - democratie et citoyennete,NaN,NaN,NaN,NaN,NaN
2739997,74053,UHJvcG9zYWw6Yzg4MWM3MzMtNDE4YS0xMWU5LWJmNTYtZm...,1-89239,Pourun juste équilibre entre représentation et...,2019-03-08,2019-03-08,NaN,False,NaN,VXNlcjo2NTY2N2ZiNC00MTgzLTExZTktYmY1Ni1mYTE2M2...,...,Y a-t-il d'autres points sur la démocratie et ...,0.0,0.0,0.0,Grand debat - democratie et citoyennete,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_elastic[df_elastic['latitude'].isna()]["authorZipCode"].value_counts().head()

In [9]:
df_elastic['source']='Grand débat - démocratie et citoyenneté'
df_elastic.to_csv("democratie_enriched.csv", encoding='utf-8', index=False)

## json writing for elasticsearch

elastic search requires a special format of json

In [10]:
df_elastic = pd.read_csv('democratie_enriched.csv', encoding='utf-8', engine='python')

In [11]:
f = open("democratie_base_pre_es.json", "w")

for row in df_elastic.iterrows():
    row[1].to_json(f, orient='index', force_ascii=True)
    f.write("\n")
f.write("\n")

1

## Ingestion in elasticsearch

In [14]:
try : 
    df_elastic['trashed'] = np.where((df_elastic['trashed'] =='True') & (df_elastic['trashed'] =='False'),True,False)
except:

with open('./democratie_base_pre_es.json', 'r') as r, open('./democratie_base_es.json', 'w') as w:    
    for num, line in enumerate(r):
        if line[0]!='[' and line[0]!=']':
            newline = '{"index":{"_id":'+ '"d_' + str(num)+'"}}'+'\n'
            if line[0]=='\n':
                newline='\n'
            elif line[-2]== ',':
                newline += line[:-2] + "\n" if "\n" in line else line[:-1]
            else:
                newline += line[:-1] + "\n" if "\n" in line else line[:-1]
        else:
            newline = ''               
        w.write(newline)
        

Then we perform the queries directly in curl to index data. 
Ex:

```curl 
curl -u username:password elasticsearch_endpoint_url/democratie/default/_bulk -H "Content-Type: application/x-ndjson" -XPOST --data-binary "@democratie_base_es_1.json"
ping 127.0.0.1 -n 6 > nul```


Tips: the `ping` part is used to force the terminal to wait between ingestions